In [1]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import torch
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType

/home/peteflo/code/modules/dense_correspondence_manipulation/utils/transformations.py:1912: UserWarning: failed to import module _transformations
  warnings.warn('failed to import module %s' % name)


In [2]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
config = utils.getDictFromYamlFilename(config_filename)
default_config = utils.get_defaults_config()


utils.set_cuda_visible_devices([0])
dce = DenseCorrespondenceEvaluation(config)
DCE = DenseCorrespondenceEvaluation



network_name = "3_caterpillar_m_background_1.0"
dcn = dce.load_network_from_config(network_name)
dcn.eval()
dataset = dcn.load_training_dataset()

setting CUDA_VISIBLE_DEVICES =  0,


/usr/local/lib/python2.7/dist-packages/torch/nn/modules/module.py:482: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  own_state[name].copy_(param)


Using SpartanDataset:
   - in train mode
   - number of scenes 17
   - total images:     5242


In [3]:
from PIL import Image
from plotting import normalize_descriptor
import time

## Usage:
## change the full_rgb_only_path
## run the full notebook

#full_rgb_only_path = "/media/peteflo/3TBbackup/dense_correspondence/pdc/logs_special/2018-05-09-15-46-59/processed/rgb_only"
full_rgb_only_path = "/media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-02-59/processed/images"
save_path = os.path.join(full_rgb_only_path, "../descriptors_only_masked")
print save_path
os.system("mkdir -p "+ save_path)

descriptor_image_stats = dcn.descriptor_image_stats

for i in sorted(os.listdir(full_rgb_only_path)):
    #print i
    start = time.time()
    if "rgb.png" not in i:
        continue

    rgb_filename = os.path.join(full_rgb_only_path, i)
    rgb_a = Image.open(rgb_filename).convert('RGB')

    # compute dense descriptors
    # This takes in a PIL image!
    rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)

    # these are Variables holding torch.FloatTensors, first grab the data, then convert to numpy
    res_a = dcn.forward_single_image_tensor(rgb_a_tensor).data.cpu().numpy()
    res_a = normalize_descriptor(res_a, descriptor_image_stats["mask_image"])
    
    # MASK
    mask_name = i.split("_rgb")[0] + "_mask.png"
    mask_filename = os.path.join(os.path.dirname(rgb_filename), "../image_masks", mask_name)
    mask = np.asarray(Image.open(mask_filename))
    mask_three_channel = np.zeros((480,640,3))
    for j in range(3):
        mask_three_channel[:,:,j] = mask
    res_a = res_a * mask_three_channel
    
    save_file_name = os.path.join(save_path, i)
#     plt.imshow(res_a)
#     plt.show()
#     break
    plt.imsave(save_file_name, res_a)
    print "forward and saving at rate", time.time()-start


# plt.imshow(res_a)
# plt.show()

/media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-02-59/processed/images/../descriptors_only_masked
000000_depth.png
000000_rgb.png


/usr/local/lib/python2.7/dist-packages/torch/nn/functional.py:1423: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


forward and saving at rate 0.765197992325
000044_depth.png
000044_rgb.png
forward and saving at rate 0.192349910736
000051_depth.png
000051_rgb.png
forward and saving at rate 0.132717132568
000057_depth.png
000057_rgb.png
forward and saving at rate 0.132025003433
000062_depth.png
000062_rgb.png
forward and saving at rate 0.124513864517
000067_depth.png
000067_rgb.png
forward and saving at rate 0.126503944397
000072_depth.png
000072_rgb.png
forward and saving at rate 0.126553058624
000077_depth.png
000077_rgb.png
forward and saving at rate 0.122163057327
000083_depth.png
000083_rgb.png
forward and saving at rate 0.127116918564
000121_depth.png
000121_rgb.png
forward and saving at rate 0.114667892456
000127_depth.png
000127_rgb.png
forward and saving at rate 0.109908103943
000132_depth.png
000132_rgb.png
forward and saving at rate 0.107307910919
000136_depth.png
000136_rgb.png
forward and saving at rate 0.108787059784
000140_depth.png
000140_rgb.png
forward and saving at rate 0.105334997

forward and saving at rate 0.117673158646
000914_depth.png
000914_rgb.png
forward and saving at rate 0.102576971054
000919_depth.png
000919_rgb.png
forward and saving at rate 0.110584974289
000923_depth.png
000923_rgb.png
forward and saving at rate 0.10163807869
000928_depth.png
000928_rgb.png
forward and saving at rate 0.0999970436096
000933_depth.png
000933_rgb.png
forward and saving at rate 0.101348876953
000939_depth.png
000939_rgb.png
forward and saving at rate 0.100683927536
000968_depth.png
000968_rgb.png
forward and saving at rate 0.11329293251
000973_depth.png
000973_rgb.png
forward and saving at rate 0.114393949509
000977_depth.png
000977_rgb.png
forward and saving at rate 0.10587477684
000981_depth.png
000981_rgb.png
forward and saving at rate 0.106611013412
000985_depth.png
000985_rgb.png
forward and saving at rate 0.105262041092
000990_depth.png
000990_rgb.png
forward and saving at rate 0.105190992355
001018_depth.png
001018_rgb.png
forward and saving at rate 0.10567784309

In [5]:
# make an rgb only dir
rgb_only_path = os.path.join(os.path.dirname(full_rgb_only_path),"rgb_only")
os.system("mkdir -p "+ rgb_only_path)
os.system("cp "+full_rgb_only_path+"/*rgb.png "+ rgb_only_path)

0

In [ ]:
# best so far
# ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -r 30 out.mp4

In [6]:
# make descriptor video
os.system("cd "+save_path+" && ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -r 30 video_descriptors.mp4")
# make rgb video
os.system("cd "+rgb_only_path+" && ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -r 30 video_rgb.mp4")

0

In [8]:
# merge the videos!

processed_path = os.path.dirname(rgb_only_path)
rgb_video = os.path.join(rgb_only_path, "video_rgb.mp4")
descriptors_video = os.path.join(save_path, "video_descriptors.mp4")

cmd = "cd "+processed_path+" && ffmpeg \
  -i "+rgb_video +" \
  -i "+descriptors_video +"\
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map [vid] \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  output.mp4"

print cmd
os.system(cmd)

cd /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-02-59/processed && ffmpeg   -i /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-02-59/processed/rgb_only/video_rgb.mp4   -i /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-02-59/processed/images/../descriptors_only_masked/video_descriptors.mp4  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]'   -map [vid]   -c:v libx264   -crf 23   -preset veryfast   output.mp4


0